
# Flowsheet Gibbs Reactor Simulation and Optimization of Steam Methane Reforming


## Learning outcomes


- Call and implement the IDAES GibbsReactor unit model
- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a flowsheet using Arcs
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Following the previous example of [Steam Methane Reformation in an Equilibrium Reactor](../../../Examples/UnitModels/Reactors/equilibrium_reactor.ipynb), this example solves the flowsheet using a Gibbs Reactor instead. The steam methane reformation example is adapted from S.Z. Abbas, V. Dupont, T. Mahmud, Kinetics study and modelling of steam methane reforming process over a NiO/Al2O3 catalyst in an adiabatic packed bed reactor. Int. J. Hydrogen Energy, 42 (2017), pp. 2889-2903. Typically, the process follows the chemical equations below:

**CH<sub>4</sub> + H<sub>2</sub>O → CO + 3H<sub>2</sub>**  
**CO + H<sub>2</sub>O → CO<sub>2</sub> + H<sub>2</sub>**

However, the GibbsReactor unit model solves the equilibrium by minimizing Gibbs free energy. Conveniently, this eliminates the need for a reaction package although a thermophysical package is still required.

The flowsheet that we will be using for this module is shown below with the stream conditions. As in the prior example, we will be processing natural gas and steam feeds of fixed composition to produce hydrogen. The process consists of a mixer M101 for the two inlet streams, a compressor to compress the feed to the reaction pressure, a heater H101 to heat the feed to the reaction temperature, and a GibbsReactor unit R101. We will use thermophysical properties following the Peng-Robinsion cubic equation of state for this flowsheet.

The state variables chosen for the property package are **total molar flows of each stream, temperature of each stream and pressure of each stream, and mole fractions of each component in each stream**. The components considered are: **CH<sub>4</sub>, H<sub>2</sub>O, CO, CO<sub>2</sub>, and H<sub>2</sub>** and the process occurs in vapor phase only. Therefore, every stream has 1 flow variable, 5 mole fraction variables, 1 temperature and 1 pressure variable. 

![](msr_flowsheet.png)



## Importing required Pyomo and IDAES components


To construct a flowsheet, we will need several components from the Pyomo and Idaes packages, as well as some utility tools to build the flowsheet. For further details on these components, please refer to the [Pyomo documentation]( https://pyomo.readthedocs.io/en/latest/).

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Feed
- Mixer
- Compressor
- Heater
- GibbsReactor
- Product

In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           TransformationFactory,
                           value,
                           units as pyunits)
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
        GenericParameterBlock)
from idaes.models.unit_models import (Feed,
                                      Mixer,
                                      Compressor,
                                      Heater,
                                      GibbsReactor,
                                      Product)

from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

## Importing required thermophysical package

As mentioned earlier, the GibbsReactor does not require a reaction package.

Let us import the following module from the IDAES library:
- natural_gas_PR as get_prop (method to get configuration dictionary)

In [2]:
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop

## Constructing the Flowsheet

Let us create a ConcreteModel and add the flowsheet block. 

In [3]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike the previous example, we do not need to add a reaction package for the reactor model to calculate results. We will use the [Modular Property Framework](https://idaes-pse.readthedocs.io/en/stable/explanations/components/property_package/index.html#generic-property-package-framework) to build a state block for the parameter dictionary.

In [4]:
thermo_props_config_dict = get_prop(components=["CH4", "H2O", "H2", "CO", "CO2"])
m.fs.thermo_params = GenericParameterBlock(default=thermo_props_config_dict)

## Adding Unit Models

Let us start adding the unit models we have imported to the flowsheet:

In [5]:
m.fs.CH4 = Feed(default={"property_package": m.fs.thermo_params})
m.fs.H2O = Feed(default={"property_package": m.fs.thermo_params})
m.fs.PROD = Product(default={"property_package": m.fs.thermo_params})
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["methane_feed", "steam_feed"]})
m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": False})
m.fs.C101 = Compressor(default={"property_package": m.fs.thermo_params})

In [6]:
m.fs.R101 = GibbsReactor(
            default={"property_package": m.fs.thermo_params,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

## Connecting Unit Models using Arcs

We have now added all the unit models we need to the flowsheet. Let us connect the unit models by defining and building the Arcs:

In [7]:
m.fs.s01 = Arc(source=m.fs.CH4.outlet, destination=m.fs.M101.methane_feed)
m.fs.s02 = Arc(source=m.fs.H2O.outlet, destination=m.fs.M101.steam_feed)
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.C101.inlet)
m.fs.s04 = Arc(source=m.fs.C101.outlet, destination=m.fs.H101.inlet)
m.fs.s05 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s06 = Arc(source=m.fs.R101.outlet, destination=m.fs.PROD.inlet)

Finally, we can use Pyomo's `TransformationFactory` to write the equality constraints on each Arc:

In [8]:
TransformationFactory("network.expand_arcs").apply_to(m)

## Adding expressions to compute operating costs

For this flowsheet, we are interested in computing hydrogen production in millions of pounds per year, as well as the total costs due to pressurizing, cooling and heating utilities:

Let us add Expressions to convert the product flow from mol/s to MM lb/year of hydrogen, and to calculate the cooling, heating and compression operating costs. The total operating cost will be the sum of the costs, expressed in \\$/year assuming 8000 operating hours per year (~10\% downtime, which is fairly common for small scale chemical plants):

In [9]:
m.fs.hyd_prod = Expression(expr=pyunits.convert(m.fs.PROD.inlet.flow_mol[0]*m.fs.PROD.inlet.mole_frac_comp[0, "H2"]
                                                *m.fs.thermo_params.H2.mw, # MW defined in properties as kg/mol
                                                to_units=pyunits.Mlb/pyunits.yr)) # converting kg/s to MM lb/year

In [10]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * (m.fs.R101.heat_duty[0]))  # the reaction is endothermic, so R101 duty is positive
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0])  # the stream must be heated to T_rxn, so H101 duty is positive
m.fs.compression_cost = Expression(expr=0.12E-5 * m.fs.C101.work_isentropic[0]) # the stream must be pressurized, so the C101 work is positive
m.fs.operating_cost = Expression(expr=(3600 * 8000 *(m.fs.heating_cost + m.fs.cooling_cost + m.fs.compression_cost)))

## Fixing feed conditions

We expect each stream to have 8 degrees of freedom, the mixer to have 0 (after both streams are accounted for), the compressor to have 2 (the pressure change and efficiency), the heater to have 1 (just the duty, since the inlet is also the outlet of M101), and the reactor to have 1 (conversion). Therefore, we have 20 degrees of freedom to specify: temperature, pressure, flow and mole fractions of all five components on both streams; compressor pressure change and efficiency; outlet heater temperature; and reactor conversion.

In [11]:
print(degrees_of_freedom(m))

20


In [12]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 20

Based the literature source, we will initialize our simulation with the following values:

In [13]:
m.fs.CH4.outlet.mole_frac_comp[0, "CH4"].fix(1)
m.fs.CH4.outlet.mole_frac_comp[0, "H2O"].fix(1E-5)
m.fs.CH4.outlet.mole_frac_comp[0, "H2"].fix(1E-5)
m.fs.CH4.outlet.mole_frac_comp[0, "CO"].fix(1E-5)
m.fs.CH4.outlet.mole_frac_comp[0, "CO2"].fix(1E-5)
m.fs.CH4.outlet.flow_mol.fix(75*pyunits.mol/pyunits.s)
m.fs.CH4.outlet.temperature.fix(298.15*pyunits.K)
m.fs.CH4.outlet.pressure.fix(1e5*pyunits.Pa)

m.fs.H2O.outlet.mole_frac_comp[0, "CH4"].fix(1E-5)
m.fs.H2O.outlet.mole_frac_comp[0, "H2O"].fix(1)
m.fs.H2O.outlet.mole_frac_comp[0, "H2"].fix(1E-5)
m.fs.H2O.outlet.mole_frac_comp[0, "CO"].fix(1E-5)
m.fs.H2O.outlet.mole_frac_comp[0, "CO2"].fix(1E-5)
m.fs.H2O.outlet.flow_mol.fix(234*pyunits.mol/pyunits.s)
m.fs.H2O.outlet.temperature.fix(373.15*pyunits.K)
m.fs.H2O.outlet.pressure.fix(1e5*pyunits.Pa)

## Fixing unit model specifications

For the initial problem, let us fix the compressor outlet pressure to 2 bar for now, the efficiency to 0.90 (a common assumption for compressor units), and the heater outlet temperature to 500 K. We will unfix these values later to optimize the flowsheet.

In [14]:
m.fs.C101.outlet.pressure.fix(pyunits.convert(2*pyunits.bar, to_units=pyunits.Pa))
m.fs.C101.efficiency_isentropic.fix(0.90)
m.fs.H101.outlet.temperature.fix(500*pyunits.K)

The GibbsReactor unit model calculates the amount of product and reactant based on the free energy minimization; therefore, we will specify a desired conversion and let the solver determine the reactor duty and heat transfer. For convenience, we will define the reactor conversion as the amount of methane that is converted.

In [15]:
m.fs.R101.conversion = Var(initialize=0.80, bounds=(0, 1), units=pyunits.dimensionless)  # fraction

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.flow_mol[0]*
    m.fs.R101.inlet.mole_frac_comp[0, "CH4"] ==
    (m.fs.R101.inlet.flow_mol[0]*m.fs.R101.inlet.mole_frac_comp[0, "CH4"] -
     m.fs.R101.outlet.flow_mol[0]*m.fs.R101.outlet.mole_frac_comp[0, "CH4"]))

m.fs.R101.conversion.fix(0.80)

In [16]:
print(degrees_of_freedom(m))

0


In [17]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 0

Finally, we need to initialize the each unit operation and propagate the outlet results in sequence to solve the flowsheet:

In [18]:
# Initialize and solve each unit operation
m.fs.CH4.initialize()
propagate_state(arc=m.fs.s01)

m.fs.H2O.initialize()
propagate_state(arc=m.fs.s02)

m.fs.M101.initialize()
propagate_state(arc=m.fs.s03)

m.fs.C101.initialize()
propagate_state(arc=m.fs.s04)

m.fs.H101.initialize()
propagate_state(arc=m.fs.s05)

m.fs.R101.initialize()
propagate_state(arc=m.fs.s06)

m.fs.PROD.initialize()

# set solver
solver = get_solver()

2022-06-01 09:50:44 [INFO] idaes.init.fs.CH4.properties: Starting initialization
2022-06-01 09:50:45 [INFO] idaes.init.fs.CH4.properties: Property initialization: optimal - Optimal Solution Found.
2022-06-01 09:50:45 [INFO] idaes.init.fs.CH4.properties: Property package initialization: optimal - Optimal Solution Found.
2022-06-01 09:50:45 [INFO] idaes.init.fs.CH4: Initialization Complete.
2022-06-01 09:50:45 [INFO] idaes.init.fs.H2O.properties: Starting initialization
2022-06-01 09:50:45 [INFO] idaes.init.fs.H2O.properties: Property initialization: optimal - Optimal Solution Found.
2022-06-01 09:50:45 [INFO] idaes.init.fs.H2O.properties: Property package initialization: optimal - Optimal Solution Found.
2022-06-01 09:50:45 [INFO] idaes.init.fs.H2O: Initialization Complete.
2022-06-01 09:50:45 [INFO] idaes.init.fs.M101.methane_feed_state: Starting initialization
2022-06-01 09:50:45 [INFO] idaes.init.fs.M101.methane_feed_state: Property initialization: optimal - Optimal Solution Found.
2

In [19]:
# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [20]:
# Check solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

## Analyze the results of the square problem


What is the total operating cost? 

In [21]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

operating cost = $39.958 million per year


In [22]:
import pytest
assert value(m.fs.operating_cost)/1E6 == pytest.approx(39.958, rel=1e-3)

For this operating cost, what conversion did we achieve of methane to hydrogen?

In [23]:
m.fs.R101.report()

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value      : Units : Fixed : Bounds
    Heat Duty : 1.7819e+07 :  watt : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                Units         Inlet     Outlet  
    Total Molar Flowrate     mole / second     309.01     429.02
    Total Mole Fraction CH4  dimensionless    0.24272   0.034965
    Total Mole Fraction H2O  dimensionless    0.75725    0.32532
    Total Mole Fraction H2   dimensionless 9.9996e-06    0.49984
    Total Mole Fraction CO   dimensionless 9.9996e-06   0.059609
    Total Mole Fraction CO2  dimensionless 9.9996e-06   0.080265
    Temperature                     kelvin     500.00     920.80
    Pressure                        pascal 2.0000e+05 2.0

In [24]:
assert value(m.fs.R101.conversion) == pytest.approx(0.800, rel=1e-3)
assert value(m.fs.R101.heat_duty[0])/1E6 == pytest.approx(17.819, rel=1e-3)
assert value(m.fs.R101.outlet.temperature[0])/1E2 == pytest.approx(9.208, rel=1e-3)

## Optimizing Hydrogen Production

Now that the flowsheet has been squared and solved, we can run a small optimization problem to determine optimal conditions for producing hydrogen. Suppose we wish to find ideal conditions for the competing reactions. The GibbsReactor does not drive equilibrium forward based on temperature, so we will see small amounts of intermediate components present in the product stream. We will allow for variable reactor temperature and pressure by freeing our heater and compressor specifications, and minimize cost to achieve 90% methane conversion. Since we assume an isentopic compressor, allowing compression will heat our feed stream and reduce or eliminate the required heater duty.

Let us declare our objective function for this problem. 

In [25]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

Now, we need to add the design constraints and unfix the decision variables as we had solved a square problem (degrees of freedom = 0) until now, as well as set bounds for the design variables (reactor outlet temperature is set by state variable bounds in property package):

In [26]:
m.fs.R101.conversion.fix(0.90)

m.fs.C101.outlet.pressure.unfix()
m.fs.C101.outlet.pressure[0].setlb(pyunits.convert(1*pyunits.bar, to_units=pyunits.Pa))  # equals inlet pressure
m.fs.C101.outlet.pressure[0].setlb(pyunits.convert(10*pyunits.bar, to_units=pyunits.Pa))  # at most, pressurize to 1 bar

m.fs.H101.outlet.temperature.unfix()
m.fs.H101.heat_duty[0].setlb(0*pyunits.J/pyunits.s)  # ensures outlet is equal to or greater than inlet temperature
m.fs.H101.outlet.temperature[0].setub(1000*pyunits.K)  # at most, heat to 1000 K

In [27]:
assert degrees_of_freedom(m) == 2


We have now defined the optimization problem and we are now ready to solve this problem. 




In [28]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [29]:
# Check for solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

In [30]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

print()
print('Compressor results')

m.fs.C101.report()

print()
print('Heater results')

m.fs.H101.report()

print()
print('Gibbs reactor results')

m.fs.R101.report()

operating cost = $107.218 million per year

Compressor results

Unit : fs.C101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value      : Units         : Fixed : Bounds
    Isentropic Efficiency :    0.90000 : dimensionless :  True : (None, None)
          Mechanical Work : 3.0334e+06 :          watt : False : (None, None)
          Pressure Change : 9.0000e+05 :        pascal : False : (None, None)
           Pressure Ratio :     10.000 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                Units         Inlet     Outlet  
    Total Molar Flowrate     mole / second     309.01     309.01
    Total Mole Fraction CH4  dimensionless    0.24272    0.24272
    Total Mole Fraction H2O  dimensionless   

In [31]:
assert value(m.fs.operating_cost)/1E6 == pytest.approx(107.218, rel=1e-3)

Display optimal values for the decision variables and design variables:

In [32]:
print('Optimal Values')
print()

print(f"C101 outlet pressure = {value(m.fs.C101.outlet.pressure[0])/1E6:0.3f} MPa")
print()

print(f"C101 outlet temperature = {value(m.fs.C101.outlet.temperature[0]):0.3f} K")
print()

print(f"H101 outlet temperature = {value(m.fs.H101.outlet.temperature[0]):0.3f} K")

print()
print(f"R101 outlet temperature = {value(m.fs.R101.outlet.temperature[0]):0.3f} K")

print()
print(f"Hydrogen produced = {value(m.fs.hyd_prod):0.3f} MM lb/year")

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")

Optimal Values

C101 outlet pressure = 1.000 MPa

C101 outlet temperature = 619.248 K

H101 outlet temperature = 619.248 K

R101 outlet temperature = 1087.385 K

Hydrogen produced = 32.070 MM lb/year

Conversion achieved = 90.0%


In [33]:
assert value(m.fs.C101.outlet.pressure[0])/1E6 == pytest.approx(1.000, rel=1e-3)
assert value(m.fs.C101.outlet.temperature[0])/100 == pytest.approx(6.19248, rel=1e-3)
assert value(m.fs.H101.outlet.temperature[0])/100 == pytest.approx(6.19248, rel=1e-3)
assert value(m.fs.R101.outlet.temperature[0])/100 == pytest.approx(10.8738, rel=1e-3)
assert value(m.fs.hyd_prod) == pytest.approx(32.070, rel=1e-3)
assert value(m.fs.R101.conversion)*100 == pytest.approx(90.0, rel=1e-3)